In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np


In [ ]:
# Load and prepare the dataset
data = pd.read_csv('data/DATA.csv', sep=";")
data['T'] = pd.to_datetime(data['T'], dayfirst=True)
data['WL_Pank'] = data['WL_Pank'] * 0.01
data['WL_Sof'] = data['WL_Sof'] * 0.01
data['WL_Bou'] = data['WL_Bou'] * 0.01
data['WL_Dou'] = data['WL_Dou'] * 0.01
data['WL_Mop'] = data['WL_Mop'] * 0.01
data['P'] = data['P'] * 0.001
data

# CV FOLD 1 #

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

# Load your dataset and prepare it as usual
# Assuming data is already loaded and preprocessed as needed

# Define X and y
X_fold1 = data[['Tot_evap', 'WL_Sof', 'WL_Bou', 'WL_Pank', 'P', 'WL_Dou', 'WTD']]  # Inputs
y_fold1 = data['WL_Mop']  # Target

# Split the data: first 60% training, middle 20% testing, last 20% validation
train_size = int(0.6 * len(X_fold1))
test_size = int(0.2 * len(X_fold1))

# The first 60% for training
X_train_fold1 = X_fold1.iloc[:train_size]
y_train_fold1 = y_fold1.iloc[:train_size]

# The next 20% for testing
X_test_fold1 = X_fold1.iloc[train_size:train_size + test_size]
y_test_fold1 = y_fold1.iloc[train_size:train_size + test_size]

# The last 20% for validation
X_val_fold1 = X_fold1.iloc[train_size + test_size:]
y_val_fold1 = y_fold1.iloc[train_size + test_size:]

# Scale the data
scaler_fold1 = StandardScaler()

# Fit the scaler on the training data and transform all datasets
X_train_scaled_fold1 = scaler_fold1.fit_transform(X_train_fold1)
X_val_scaled_fold1 = scaler_fold1.transform(X_val_fold1)
X_test_scaled_fold1 = scaler_fold1.transform(X_test_fold1)

# Check the results (optional)
print("Training set:", X_train_fold1.shape, y_train_fold1.shape)
print("Testing set:", X_test_fold1.shape, y_test_fold1.shape)
print("Validation set:", X_val_fold1.shape, y_val_fold1.shape)


In [ ]:
# Correcting the validation and test sets
train_dates_fold1 = data['T'].iloc[X_train_fold1.index] 

test_dates_fold1 = data['T'].iloc[X_test_fold1.index]     

val_dates_fold1 = data['T'].iloc[X_val_fold1.index]     

# Print the first few dates to check after the swap
print("Training Dates:\n", train_dates_fold1)

print("\nTesting Dates:\n", test_dates_fold1)

print("\nValidation Dates:\n", val_dates_fold1)


In [ ]:
import matplotlib.pyplot as plt
# Plot the training, testing, and validation data for Fold1
Tim = data['T']

plt.figure(figsize=(12, 4))

# Plot training data
plt.plot(Tim.iloc[X_train_fold1.index], y_train_fold1, 'g', label='Training')

# Plot testing data (which comes after training)
plt.plot(Tim.iloc[X_test_fold1.index], y_test_fold1, 'b', label='Testing')

# Plot validation data (which comes after testing)
plt.plot(Tim.iloc[X_val_fold1.index], y_val_fold1, 'r', label='Validation')

# Add labels and legend
plt.ylabel('Water Level (WL_Mop)')
plt.legend(fontsize=8)
plt.show()


In [ ]:


# Define the NSE function
def nse(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2))

# Function to train the model with predefined hyperparameters and evaluate using NSE on both validation and test sets
def train_and_evaluate(model, X_train, y_train, X_val, y_val, X_test, y_test):
    model.fit(X_train, y_train)
    
    # Predictions and NSE for validation set
    val_pred = model.predict(X_val)
    val_nse = nse(y_val, val_pred)
    
    # Predictions and NSE for test set
    test_pred = model.predict(X_test)
    test_nse = nse(y_test, test_pred)
    
    return model, val_nse, test_nse

# Best parameters for each model
rf_params = {
    'n_estimators': 500,
    'max_depth': 10,
    'min_samples_split': 3,
    'min_samples_leaf': 3
}

gb_params = {
    'n_estimators': 104,
    'learning_rate': 0.06308188348933369,
    'max_depth': 3,
    'subsample': 0.721266023411284
}

svm_params = {
    'C': 0.1,
    'epsilon': 0.09393301263712334,
    'gamma': 0.09783812402611333
}

xgb_params = {
    'n_estimators': 500,
    'learning_rate': 0.05103704917722505,
    'max_depth': 9,
    'subsample': 0.8782745247955278,
    'colsample_bytree': 0.6
}

# Instantiate models with the best hyperparameters
models_fold1 = {
    'RandomForest': RandomForestRegressor(**rf_params),
    'GradientBoosting': GradientBoostingRegressor(**gb_params),
    'SVM': SVR(**svm_params),
    'XGB': XGBRegressor(**xgb_params)
}

# Train and evaluate models using the NSE metric on both validation and test sets
results_fold1 = {}
for model_name, model in models_fold1.items():
    trained_model, val_nse, test_nse = train_and_evaluate(
        model, X_train_scaled_fold1, y_train_fold1, X_val_scaled_fold1, y_val_fold1, X_test_scaled_fold1, y_test_fold1
    )
    results_fold1[model_name] = {
        'Model': trained_model,
        'NSE_Val': val_nse,
        'NSE_Test': test_nse
    }

# Print results
for model_name, result in results_fold1.items():
    print(f"{model_name} - NSE Validation Score: {result['NSE_Val']:.2f}, NSE Test Score: {result['NSE_Test']:.2f}")

In [ ]:
# Define the evaluation function to calculate RMSE, MAPE, MAE, NSE, and R²
def evaluate_model(model, X, y):
    predictions = model.predict(X)
    rmse = mean_squared_error(y, predictions, squared=False)  # Root Mean Squared Error
    mae = mean_absolute_error(y, predictions)                 # Mean Absolute Error
    mape = mean_absolute_percentage_error(y, predictions)     # Mean Absolute Percentage Error
    r2 = r2_score(y, predictions)                             # R² Score
    nse = 1 - (mean_squared_error(y, predictions) / mean_squared_error(y, [y.mean()]*len(y)))  # Nash-Sutcliffe Efficiency
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'NSE': nse, 'R²': r2}

# Evaluate the models on the test set
test_metrics_fold1 = {name: evaluate_model(model, X_test_scaled_fold1, y_test_fold1) for name, model in models_fold1.items()}

# Evaluate the models on the validation set
val_metrics_fold1 = {name: evaluate_model(model, X_val_scaled_fold1, y_val_fold1) for name, model in models_fold1.items()}

# Convert the results into DataFrames for easier visualization
test_metrics_df_fold1 = pd.DataFrame(test_metrics_fold1).T
val_metrics_df_fold1 = pd.DataFrame(val_metrics_fold1).T

# Combine both test and validation metrics into a single DataFrame
combined_metrics_df_fold1 = pd.concat([test_metrics_df_fold1.add_suffix('_Test'), val_metrics_df_fold1.add_suffix('_Val')], axis=1)

# Display the combined metrics
print(combined_metrics_df_fold1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
# Plot observed vs. predicted values for test and validation sets
plt.rcParams.update({'font.size': 16})
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(16, 10), sharey=True)
new_titles = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']
model_titles = ['RF', 'GB', 'SVM', 'XGB']

# Loop over models and plot for both test and validation sets
for i, (model_name, model) in enumerate(models_fold1.items()):
    # Test set plot (first row)
    ax_test = axes[0, i]
    test_predictions = model.predict(X_test_scaled_fold1)
    ax_test.scatter(y_test_fold1, test_predictions, alpha=0.4, color='blue', marker='o', s=10)
    ax_test.plot([y_test_fold1.min(), y_test_fold1.max()], [y_test_fold1.min(), y_test_fold1.max()], 'r--', lw=2)
    r2_test = r2_score(y_test_fold1, test_predictions)
    ax_test.text(0.95, 0.05, new_titles[i], horizontalalignment='right', verticalalignment='bottom', transform=ax_test.transAxes, fontsize=16, color='black')
    ax_test.text(0.05, 0.95, f'R²={r2_test:.2f}', transform=ax_test.transAxes, verticalalignment='top', color='blue', fontsize=16)
    ax_test.set_title(f'{model_titles[i]}', fontsize=16)
    if i == 0:
        ax_test.set_ylabel('Predicted water level (m)', fontsize=16)

    # Validation set plot (second row)
    ax_val = axes[1, i]
    val_predictions = model.predict(X_val_scaled_fold1)
    ax_val.scatter(y_val_fold1, val_predictions, alpha=0.4, color='black', marker='o', s=10)
    ax_val.plot([y_val_fold1.min(), y_val_fold1.max()], [y_val_fold1.min(), y_val_fold1.max()], 'r--', lw=2)
    r2_val = r2_score(y_val_fold1, val_predictions)
    ax_val.text(0.95, 0.05, new_titles[i + 4], horizontalalignment='right', verticalalignment='bottom', transform=ax_val.transAxes, fontsize=16, color='black')
    ax_val.text(0.05, 0.95, f'R²={r2_val:.2f}', transform=ax_val.transAxes, verticalalignment='top', color='black', fontsize=16)
    ax_val.set_xlabel('Observed water level (m)', fontsize=16)
    if i == 0:
        ax_val.set_ylabel('Predicted water level (m)', fontsize=16)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("Observed_vs_Predicted_Test_Validation_Separate_Panels_Fold1.png", format='png', dpi=300)
plt.show()

# Plot evaluation metrics
import numpy as np
from matplotlib.ticker import FuncFormatter

# Define a function to format the ticks to one decimal place
def one_decimal(x, pos):
    return f'{x:.1f}'

# Example metrics for bar plots
models = ['RF', 'GB', 'SVM', 'XGB']
bar_width = 0.35
rmse_test = combined_metrics_df_fold1['RMSE_Test']
rmse_val = combined_metrics_df_fold1['RMSE_Val']
mae_test = combined_metrics_df_fold1['MAE_Test']
mae_val = combined_metrics_df_fold1['MAE_Val']
mape_test = combined_metrics_df_fold1['MAPE_Test']
mape_val = combined_metrics_df_fold1['MAPE_Val']
nse_test = combined_metrics_df_fold1['NSE_Test']
nse_val = combined_metrics_df_fold1['NSE_Val']

plt.rcParams.update({'font.size': 16})
fig, axs = plt.subplots(1, 4, figsize=(14, 4))

# RMSE Bar Chart
axs[0].bar(np.arange(len(models)), rmse_test, color='blue', width=bar_width, label='Test')
axs[0].bar(np.arange(len(models)) + bar_width, rmse_val, color='black', width=bar_width, label='Val')
axs[0].set_ylabel('RMSE (m)')
axs[0].set_ylim(0.8, 1.8)
axs[0].set_xticks(np.arange(len(models)) + bar_width / 2)
axs[0].set_xticklabels(models)
axs[0].text(0.95, 0.95, '(i)', transform=axs[0].transAxes, verticalalignment='top', horizontalalignment='right')
axs[0].legend(loc='upper left', ncol=2, fontsize=10)

# MAE Bar Chart
axs[1].bar(np.arange(len(models)), mae_test, color='blue', width=bar_width, label='Test')
axs[1].bar(np.arange(len(models)) + bar_width, mae_val, color='black', width=bar_width, label='Val')
axs[1].set_ylabel('MAE (m)')
axs[1].set_ylim(0.6, 1.15)
axs[1].set_xticks(np.arange(len(models)) + bar_width / 2)
axs[1].set_xticklabels(models)
axs[1].text(0.95, 0.95, '(j)', transform=axs[1].transAxes, verticalalignment='top', horizontalalignment='right')

# MAPE Bar Chart
axs[2].bar(np.arange(len(models)), mape_test, color='blue', width=bar_width, label='Test')
axs[2].bar(np.arange(len(models)) + bar_width, mape_val, color='black', width=bar_width, label='Val')
axs[2].set_ylabel('MAPE')
axs[2].set_ylim(0.4, 0.82)
axs[2].yaxis.set_major_formatter(FuncFormatter(one_decimal))
axs[2].set_xticks(np.arange(len(models)) + bar_width / 2)
axs[2].set_xticklabels(models)
axs[2].text(0.95, 0.95, '(k)', transform=axs[2].transAxes, verticalalignment='top', horizontalalignment='right')

# NSE Bar Chart
axs[3].bar(np.arange(len(models)), nse_test, color='blue', width=bar_width, label='Test')
axs[3].bar(np.arange(len(models)) + bar_width, nse_val, color='black', width=bar_width, label='Val')
axs[3].set_ylabel('NSE')
axs[3].set_ylim(0.3, 0.8)
axs[3].yaxis.set_major_formatter(FuncFormatter(one_decimal))
axs[3].set_xticks(np.arange(len(models)) + bar_width / 2)
axs[3].set_xticklabels(models)
axs[3].text(0.95, 0.95, '(l)', transform=axs[3].transAxes, verticalalignment='top', horizontalalignment='right')

plt.tight_layout()
plt.savefig("Metrics_Test_Val_Combined_Fold1.png", format='png', dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the feature names based on your input features
feature_names = X_fold1.columns

# Setup the plotting for feature importance
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 3), sharey=True)

new_titles = ['(m)', '(n)', '(o)']
model_titles = ['RF', 'GB', 'XGB']

# Plot feature importance for each model for validation set
model_idx = 0

for model_name, model in models_fold1.items():
    if model_name in ['RandomForest', 'GradientBoosting', 'XGB']:
        feature_importance = model.feature_importances_
        ax_val = axes[model_idx]
        ax_val.barh(feature_names, feature_importance, color='blue')
        ax_val.set_title(f'{model_titles[model_idx]}', fontsize=16)
        ax_val.text(0.95, 0.95, new_titles[model_idx], transform=ax_val.transAxes, fontsize=16, ha='right', va='top')
        model_idx += 1

plt.tight_layout()
plt.savefig("Feature_Importance_fold1.png", format='png', dpi=300)
plt.show()


In [ ]:
# Create the comparison DataFrame for the test set with dates, observed values, and predicted values
df1_comparison_test = pd.DataFrame({
    'Date': data['T'].iloc[X_test_fold1.index],  # The Date column based on the test set
    'Observed': y_test_fold1,  # The observed values (ground truth)
    'Predicted SVM': models_fold1['SVM'].predict(X_test_scaled_fold1),  # Predictions from SVM model
    'Predicted RF': models_fold1['RandomForest'].predict(X_test_scaled_fold1),  # Predictions from Random Forest
    'Predicted XGB': models_fold1['XGB'].predict(X_test_scaled_fold1),  # Predictions from XGBoost
    'Predicted GB': models_fold1['GradientBoosting'].predict(X_test_scaled_fold1)  # Predictions from Gradient Boosting
})

# Create the comparison DataFrame for the validation set with dates, observed values, and predicted values
df1_comparison_val = pd.DataFrame({
    'Date': data['T'].iloc[X_val_fold1.index],  # The Date column based on the validation set
    'Observed': y_val_fold1,  # The observed values (ground truth)
    'Predicted SVM': models_fold1['SVM'].predict(X_val_scaled_fold1),  # Predictions from SVM model
    'Predicted RF': models_fold1['RandomForest'].predict(X_val_scaled_fold1),  # Predictions from Random Forest
    'Predicted XGB': models_fold1['XGB'].predict(X_val_scaled_fold1),  # Predictions from XGBoost
    'Predicted GB': models_fold1['GradientBoosting'].predict(X_val_scaled_fold1)  # Predictions from Gradient Boosting
})

# Display the comparison DataFrames
print("Test Set Comparison:\n", df1_comparison_test.head())
print("\nValidation Set Comparison:\n", df1_comparison_val.head())

In [ ]:
# Define the models and corresponding colors for plots
models = ['RF', 'GB', 'SVM', 'XGB']
colors = ['blue', 'green', 'red', 'purple']

# Create a figure with a 4x2 grid layout (4 models, test and validation columns)
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(12, 16), sharey=True)

# Loop over each model and create the corresponding plot for test (left column) and validation (right column)
for idx, model in enumerate(models):
    # Test set (first column)
    axes[idx, 0].plot(df1_comparison_test['Date'], df1_comparison_test['Observed'], label='Observed', color='black', linestyle='-', linewidth=1.5)
    axes[idx, 0].plot(df1_comparison_test['Date'], df1_comparison_test[f'Predicted {model}'], label=f'{model}', color=colors[idx], linestyle='-', linewidth=1.5)
    
    # Set limits for test set plot
    axes[idx, 0].set_xlim([df1_comparison_test['Date'].min(), df1_comparison_test['Date'].max()])  # Limit to test dataset only
    axes[idx, 0].set_ylim(0, 7.5)
    axes[idx, 0].set_title(f'Test - {model}', fontsize=14)

    # Set y-axis label for the first column
    axes[idx, 0].set_ylabel('Water Level (m)', fontsize=14)
    
    # Remove x-ticks for all except the last test plot
    if idx < 3:
        axes[idx, 0].tick_params(axis='x', which='both', bottom=False, labelbottom=False)

    # Rotate x-axis dates by 45 degrees for the last test plot only
    if idx == 3:
        axes[idx, 0].tick_params(axis='x', labelrotation=45)

    # Add legend for test set
    axes[idx, 0].legend(loc='upper left', fontsize=14, framealpha=0, ncol=2)
    
    # Annotate each subplot (test)
    axes[idx, 0].annotate(f'(a{idx + 1})', xy=(0.95, 0.95), xycoords='axes fraction', fontsize=14, ha='right', va='top')

    # Validation set (second column)
    axes[idx, 1].plot(df1_comparison_val['Date'], df1_comparison_val['Observed'], label='Observed', color='black', linestyle='-', linewidth=1.5)
    axes[idx, 1].plot(df1_comparison_val['Date'], df1_comparison_val[f'Predicted {model}'], label=f'{model}', color=colors[idx], linestyle='-', linewidth=1.5)
    
    # Set limits for validation set plot
    axes[idx, 1].set_xlim([df1_comparison_val['Date'].min(), df1_comparison_val['Date'].max()])  # Limit to validation dataset only
    axes[idx, 1].set_ylim(0, 7.5)
    axes[idx, 1].set_title(f'Validation - {model}', fontsize=14)

    # Remove x-ticks for all except the last validation plot
    if idx < 3:
        axes[idx, 1].tick_params(axis='x', which='both', bottom=False, labelbottom=False)

    # Rotate x-axis dates by 45 degrees for the last validation plot only
    if idx == 3:
        axes[idx, 1].tick_params(axis='x', labelrotation=45)

    # Add legend for validation set
    axes[idx, 1].legend(loc='upper left', fontsize=14, framealpha=0, ncol=2)
    
    # Annotate each subplot (validation)
    axes[idx, 1].annotate(f'(b{idx + 1})', xy=(0.95, 0.95), xycoords='axes fraction', fontsize=14, ha='right', va='top')

# Adjust layout to avoid overlap
plt.tight_layout()

# Save the plot as a high-resolution PNG file
plt.savefig("Observed_vs_Predicted_Test_Validation_Panel_fold1.png", format='png', dpi=300)

# Display the plot
plt.show()

# CV FOLD 2

In [ ]:
# Define X and y for fold2
X_fold2 = data[['Tot_evap', 'WL_Sof', 'WL_Bou', 'WL_Pank', 'P', 'WL_Dou', 'WTD']]  # Inputs
y_fold2 = data['WL_Mop']  # Target

# Split the data: first 20% testing, middle 60% training, last 20% validation
train_size_fold2 = int(0.6 * len(X_fold2))
test_size_fold2 = int(0.2 * len(X_fold2))

# The first 20% for testing
X_test_fold2 = X_fold2.iloc[:test_size_fold2]
y_test_fold2 = y_fold2.iloc[:test_size_fold2]

# The middle 60% for training
X_train_fold2 = X_fold2.iloc[test_size_fold2:test_size_fold2 + train_size_fold2]
y_train_fold2 = y_fold2.iloc[test_size_fold2:test_size_fold2 + train_size_fold2]

# The last 20% for validation
X_val_fold2 = X_fold2.iloc[test_size_fold2 + train_size_fold2:]
y_val_fold2 = y_fold2.iloc[test_size_fold2 + train_size_fold2:]

# Scale the data
scaler_fold2 = StandardScaler()

# Fit the scaler on the training data and transform all datasets
X_train_scaled_fold2 = scaler_fold2.fit_transform(X_train_fold2)
X_val_scaled_fold2 = scaler_fold2.transform(X_val_fold2)
X_test_scaled_fold2 = scaler_fold2.transform(X_test_fold2)

# Correcting the validation and test sets
train_dates_fold2 = data['T'].iloc[X_train_fold2.index]
test_dates_fold2 = data['T'].iloc[X_test_fold2.index]
val_dates_fold2 = data['T'].iloc[X_val_fold2.index]

# Print the first few dates to check after the swap
print("Training Dates:\n", train_dates_fold2)
print("\nTesting Dates:\n", test_dates_fold2)
print("\nValidation Dates:\n", val_dates_fold2)

In [ ]:
# Plot the training, testing, and validation data
Tim = data['T']

plt.figure(figsize=(12, 4))

# Plot training data
plt.plot(Tim.iloc[X_train_fold2.index], y_train_fold2, 'g', label='Training')

# Plot testing data (which comes after training)
plt.plot(Tim.iloc[X_test_fold2.index], y_test_fold2, 'b', label='Testing')

# Plot validation data (which comes after testing)
plt.plot(Tim.iloc[X_val_fold2.index], y_val_fold2, 'r', label='Validation')

plt.ylabel('Water Level (WL_Mop)')
plt.legend(fontsize=8)
plt.show()

In [ ]:
# Define the best hyperparameters for each model
best_hyperparams = {
    'RF': {
        'n_estimators': 100,
        'max_depth': 16,
        'min_samples_split': 8,
        'min_samples_leaf': 1
    },
    'GB': {
        'n_estimators': 181,
        'learning_rate': 0.01171817739833397,
        'max_depth': 3,
        'subsample': 0.6
    },
    'SVM': {
        'C': 328.3163943534423,
        'epsilon': 0.01,
        'gamma': 0.002132966718896371
    },
    'XGB': {
        'n_estimators': 121,
        'learning_rate': 0.03925201734171761,
        'max_depth': 3,
        'subsample': 1.0,
        'colsample_bytree': 0.6
    }
}

# Define model instances with the best hyperparameters
models = {
    'RF': RandomForestRegressor(**best_hyperparams['RF']),
    'GB': GradientBoostingRegressor(**best_hyperparams['GB']),
    'SVM': SVR(**best_hyperparams['SVM']),
    'XGB': XGBRegressor(**best_hyperparams['XGB'])
}

# Evaluate the models on the test and validation sets
def evaluate_model(model, X, y):
    predictions = model.predict(X)
    rmse = mean_squared_error(y, predictions, squared=False)  # Root Mean Squared Error
    mae = mean_absolute_error(y, predictions)                 # Mean Absolute Error
    mape = mean_absolute_percentage_error(y, predictions)     # Mean Absolute Percentage Error
    r2 = r2_score(y, predictions)                             # R² Score
    nse_score = nse(y, predictions)                           # Nash-Sutcliffe Efficiency
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'NSE': nse_score, 'R²': r2}

# Evaluate the models on both test and validation sets
evaluation_results_test = {}
evaluation_results_val = {}

for model_name, model in models.items():
    # Train the model
    model.fit(X_train_scaled_fold2, y_train_fold2)
    
    # Evaluate on test set
    evaluation_results_test[model_name] = evaluate_model(model, X_test_scaled_fold2, y_test_fold2)
    
    # Evaluate on validation set
    evaluation_results_val[model_name] = evaluate_model(model, X_val_scaled_fold2, y_val_fold2)

# Print the evaluation results for the test set
print("\nEvaluation results for the Test Set:")
for model_name, result in evaluation_results_test.items():
    print(f"\n{model_name}:")
    for metric, value in result.items():
        print(f"{metric}: {value:.2f}")

# Print the evaluation results for the validation set
print("\nEvaluation results for the Validation Set:")
for model_name, result in evaluation_results_val.items():
    print(f"\n{model_name}:")
    for metric, value in result.items():
        print(f"{metric}: {value:.2f}")

In [ ]:
# Update font size
plt.rcParams.update({'font.size': 16})

# Create the subplots with a 2x4 grid layout (for 8 subplots total: 4 for test and 4 for validation)
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(16, 10), sharey=True)

# Define the titles for each subplot
new_titles = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']
model_titles = ['RF', 'GB', 'SVM', 'XGB']  # Titles for each model

# Fit each model before making predictions
for model_name, model in models.items():
    model.fit(X_train_scaled_fold2, y_train_fold2)  # Train the model on training data

# Loop over models and plot for both test and validation sets in separate panels
for i, (model_name, model) in enumerate(models.items()):
    # Test set plot (first row)
    ax_test = axes[0, i]
    test_predictions = model.predict(X_test_scaled_fold2)
    
    # Scatter plot of observed vs predicted values for test set
    ax_test.scatter(y_test_fold2, test_predictions, alpha=0.4, color='blue', marker='o', s=10)
    ax_test.plot([y_test_fold2.min(), y_test_fold2.max()], [y_test_fold2.min(), y_test_fold2.max()], 'r--', lw=2)  # Line for perfect predictions

    # Add R² score for test set
    r2_test = r2_score(y_test_fold2, test_predictions)
    ax_test.text(0.95, 0.05, new_titles[i], horizontalalignment='right', verticalalignment='bottom', transform=ax_test.transAxes, fontsize=16, color='black')
    ax_test.text(0.05, 0.95, f'R²={r2_test:.2f}', transform=ax_test.transAxes, verticalalignment='top', color='blue', fontsize=16)

    # Set labels and title for each test subplot
    ax_test.set_title(f'{model_titles[i]}', fontsize=16)
    if i == 0:
        ax_test.set_ylabel('Predicted water level (m)', fontsize=16)

    # Validation set plot (second row)
    ax_val = axes[1, i]
    val_predictions = model.predict(X_val_scaled_fold2)
    
    # Scatter plot of observed vs predicted values for validation set
    ax_val.scatter(y_val_fold2, val_predictions, alpha=0.4, color='black', marker='o', s=10)
    ax_val.plot([y_val_fold2.min(), y_val_fold2.max()], [y_val_fold2.min(), y_val_fold2.max()], 'r--', lw=2)  # Line for perfect predictions

    # Add R² score for validation set
    r2_val = r2_score(y_val_fold2, val_predictions)
    ax_val.text(0.95, 0.05, new_titles[i + 4], horizontalalignment='right', verticalalignment='bottom', transform=ax_val.transAxes, fontsize=16, color='black')
    ax_val.text(0.05, 0.95, f'R²={r2_val:.2f}', transform=ax_val.transAxes, verticalalignment='top', color='black', fontsize=16)

    # Set labels and title for each validation subplot
    ax_val.set_xlabel('Observed water level (m)', fontsize=16)
    if i == 0:
        ax_val.set_ylabel('Predicted water level (m)', fontsize=16)

# Adjust layout to avoid overlap
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Add space for the suptitle

# Save the figure with 300 DPI
plt.savefig("Observed_vs_Predicted_Test_Validation_Separate_Panels_Fold2.png", format='png', dpi=300)

# Display the plot
plt.show()

# Define the models and their evaluation results from your table
models = ['RF', 'GB', 'SVM', 'XGB']
bar_width = 0.35

# Test Set results (from your table)
rmse_test = [0.39, 0.46, 0.37, 0.38]
mae_test = [0.31, 0.40, 0.28, 0.31]
mape_test = [0.24, 0.36, 0.20, 0.24]
nse_test = [0.93, 0.91, 0.94, 0.94]

# Validation Set results (from your table)
rmse_val = [0.41, 0.66, 0.52, 0.54]
mae_val = [0.31, 0.54, 0.41, 0.42]
mape_val = [0.20, 0.36, 0.29, 0.27]
nse_val = [0.96, 0.90, 0.94, 0.93]

# Define a function to format ticks to one decimal place
def one_decimal(x, pos):
    return f'{x:.1f}'

# Create a subplot grid of 1x4 for RMSE, MAE, MAPE, and NSE (test and val combined)
plt.rcParams.update({'font.size': 16})
fig, axs = plt.subplots(1, 4, figsize=(15, 4))

# RMSE Bar Chart (Test and Val combined)
axs[0].bar(np.arange(len(models)), rmse_test, color='blue', width=bar_width, label='Test')
axs[0].bar(np.arange(len(models)) + bar_width, rmse_val, color='black', width=bar_width, label='Val')
axs[0].set_ylabel('RMSE (m)')
axs[0].set_ylim(0.3, 0.75)  # Adjust the y-axis range based on the RMSE values
axs[0].set_xticks(np.arange(len(models)) + bar_width / 2)
axs[0].set_xticklabels(models)
axs[0].text(0.95, 0.95, '(i)', transform=axs[0].transAxes, verticalalignment='top', horizontalalignment='right')
axs[0].legend(loc='upper left', ncol=2, fontsize=10)

# MAE Bar Chart (Test and Val combined)
axs[1].bar(np.arange(len(models)), mae_test, color='blue', width=bar_width, label='Test')
axs[1].bar(np.arange(len(models)) + bar_width, mae_val, color='black', width=bar_width, label='Val')
axs[1].set_ylabel('MAE (m)')
axs[1].set_ylim(0.2, 0.6)  # Adjust the y-axis range based on the MAE values
axs[1].set_xticks(np.arange(len(models)) + bar_width / 2)
axs[1].set_xticklabels(models)
axs[1].text(0.95, 0.95, '(j)', transform=axs[1].transAxes, verticalalignment='top', horizontalalignment='right')

# MAPE Bar Chart (Test and Val combined)
axs[2].bar(np.arange(len(models)), mape_test, color='blue', width=bar_width, label='Test')
axs[2].bar(np.arange(len(models)) + bar_width, mape_val, color='black', width=bar_width, label='Val')
axs[2].set_ylabel('MAPE')
axs[2].set_ylim(0.1, 0.4)  # Adjust the y-axis range based on the MAPE values
axs[2].yaxis.set_major_formatter(FuncFormatter(one_decimal))
axs[2].set_xticks(np.arange(len(models)) + bar_width / 2)
axs[2].set_xticklabels(models)
axs[2].text(0.95, 0.95, '(k)', transform=axs[2].transAxes, verticalalignment='top', horizontalalignment='right')

# NSE Bar Chart (Test and Val combined)
axs[3].bar(np.arange(len(models)), nse_test, color='blue', width=bar_width, label='Test')
axs[3].bar(np.arange(len(models)) + bar_width, nse_val, color='black', width=bar_width, label='Val')
axs[3].set_ylabel('NSE')
axs[3].set_ylim(0.7, 1)  # Adjust the y-axis range based on the NSE values
axs[3].yaxis.set_major_formatter(FuncFormatter(one_decimal))
axs[3].set_xticks(np.arange(len(models)) + bar_width / 2)
axs[3].set_xticklabels(models)
axs[3].text(0.95, 0.95, '(l)', transform=axs[3].transAxes, verticalalignment='top', horizontalalignment='right')

# Adjust the layout and display the plot
plt.tight_layout()
plt.savefig("Metrics_Test_Val_Combined_Fold2.png", format='png', dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

# Define the models based on provided best hyperparameters
models_fold1 = {
    'RF': RandomForestRegressor(n_estimators=100, max_depth=16, min_samples_split=8, min_samples_leaf=1),
    'GB': GradientBoostingRegressor(n_estimators=181, learning_rate=0.01171817739833397, max_depth=3, subsample=0.6),
    'XGB': XGBRegressor(n_estimators=121, learning_rate=0.03925201734171761, max_depth=3, subsample=1.0, colsample_bytree=0.6)
}

# Fit models and collect feature importances
feature_importances = {}
for model_name, model in models_fold1.items():
    model.fit(X_train_scaled_fold2, y_train_fold2)  # Train each model
    if hasattr(model, 'feature_importances_'):
        feature_importances[model_name] = model.feature_importances_

plt.rcParams.update({'font.size': 12})
# Setup for plotting
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 3), sharey=True)
new_titles = ['(m)', '(n)', '(o)']
model_titles = ['RF', 'GB', 'XGB']

# Plot feature importance for each model
model_idx = 0
for model_name, importances in feature_importances.items():
    ax = axes[model_idx]
    ax.barh(X_train_fold2.columns, importances, color='blue')
    ax.set_title(model_titles[model_idx], fontsize=12)
    ax.text(0.95, 0.95, new_titles[model_idx], transform=ax.transAxes, fontsize=12, ha='right', va='top')
    model_idx += 1

plt.tight_layout()
plt.savefig("Feature_Importance_fold2.png", format='png', dpi=300)
plt.show()


In [ ]:
models = {
    'RF': RandomForestRegressor(**best_hyperparams['RF']),
    'GB': GradientBoostingRegressor(**best_hyperparams['GB']),
    'SVM': SVR(**best_hyperparams['SVM']),
    'XGB': XGBRegressor(**best_hyperparams['XGB'])
}

# Fit all models on the training data
for model_name, model in models.items():
    model.fit(X_train_scaled_fold2, y_train_fold2)  # Ensure all models are trained

# Create the comparison DataFrame for the test set with dates, observed values, and predicted values
df2_comparison_test = pd.DataFrame({
    'Date': data['T'].iloc[X_test_fold2.index],  # The Date column based on the test set
    'Observed': y_test_fold2,  # The observed values (ground truth)
    'Predicted SVM': models['SVM'].predict(X_test_scaled_fold2),  # Predictions from SVM model
    'Predicted RF': models['RF'].predict(X_test_scaled_fold2),  # Predictions from Random Forest
    'Predicted XGB': models['XGB'].predict(X_test_scaled_fold2),  # Predictions from XGBoost
    'Predicted GB': models['GB'].predict(X_test_scaled_fold2)  # Predictions from Gradient Boosting
})

# Create the comparison DataFrame for the validation set with dates, observed values, and predicted values
df2_comparison_val = pd.DataFrame({
    'Date': data['T'].iloc[X_val_fold2.index],  # The Date column based on the validation set
    'Observed': y_val_fold2,  # The observed values (ground truth)
    'Predicted SVM': models['SVM'].predict(X_val_scaled_fold2),  # Predictions from SVM model
    'Predicted RF': models['RF'].predict(X_val_scaled_fold2),  # Predictions from Random Forest
    'Predicted XGB': models['XGB'].predict(X_val_scaled_fold2),  # Predictions from XGBoost
    'Predicted GB': models['GB'].predict(X_val_scaled_fold2)  # Predictions from Gradient Boosting
})

# Display the comparison DataFrames
print("Test Set Comparison:\n", df2_comparison_test.head())
print("\nValidation Set Comparison:\n", df2_comparison_val.head())

In [ ]:
# Define the models and corresponding colors for plots
model_names = ['RF', 'GB', 'SVM', 'XGB']
colors = ['blue', 'green', 'red', 'purple']

# Create a figure with a 4x2 grid layout (4 models, test and validation columns)
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(12, 16), sharey=True)

# Loop over each model and create the corresponding plot for test (left column) and validation (right column)
for idx, model in enumerate(model_names):
    # Test set (first column)
    axes[idx, 0].plot(df2_comparison_test['Date'], df2_comparison_test['Observed'], label='Observed', color='black', linestyle='-', linewidth=1.5)
    axes[idx, 0].plot(df2_comparison_test['Date'], df2_comparison_test[f'Predicted {model}'], label=f'{model}', color=colors[idx], linestyle='-', linewidth=1.5)
    
    # Set limits for test set plot
    axes[idx, 0].set_xlim([df2_comparison_test['Date'].min(), df2_comparison_test['Date'].max()])  # Limit to test dataset only
    axes[idx, 0].set_ylim(0, 7.5)
    axes[idx, 0].set_title(f'Test - {model}', fontsize=14)

    # Set y-axis label for the first column
    axes[idx, 0].set_ylabel('Water Level (m)', fontsize=14)
    
    # Remove x-ticks for all except the last test plot
    if idx < 3:
        axes[idx, 0].tick_params(axis='x', which='both', bottom=False, labelbottom=False)

    # Rotate x-axis dates by 45 degrees for the last test plot only
    if idx == 3:
        axes[idx, 0].tick_params(axis='x', labelrotation=45)

    # Add legend for test set
    axes[idx, 0].legend(loc='upper left', fontsize=14, framealpha=0, ncol=2)
    
    # Annotate each subplot (test)
    axes[idx, 0].annotate(f'(a{idx + 1})', xy=(0.95, 0.95), xycoords='axes fraction', fontsize=14, ha='right', va='top')

    # Validation set (second column)
    axes[idx, 1].plot(df2_comparison_val['Date'], df2_comparison_val['Observed'], label='Observed', color='black', linestyle='-', linewidth=1.5)
    axes[idx, 1].plot(df2_comparison_val['Date'], df2_comparison_val[f'Predicted {model}'], label=f'{model}', color=colors[idx], linestyle='-', linewidth=1.5)
    
    # Set limits for validation set plot
    axes[idx, 1].set_xlim([df2_comparison_val['Date'].min(), df2_comparison_val['Date'].max()])  # Limit to validation dataset only
    axes[idx, 1].set_ylim(0, 7.5)
    axes[idx, 1].set_title(f'Validation - {model}', fontsize=14)

    # Remove x-ticks for all except the last validation plot
    if idx < 3:
        axes[idx, 1].tick_params(axis='x', which='both', bottom=False, labelbottom=False)

    # Rotate x-axis dates by 45 degrees for the last validation plot only
    if idx == 3:
        axes[idx, 1].tick_params(axis='x', labelrotation=45)

    # Add legend for validation set
    axes[idx, 1].legend(loc='upper left', fontsize=14, framealpha=0, ncol=2)
    
    # Annotate each subplot (validation)
    axes[idx, 1].annotate(f'(b{idx + 1})', xy=(0.95, 0.95), xycoords='axes fraction', fontsize=14, ha='right', va='top')

# Adjust layout to avoid overlap
plt.tight_layout()

# Save the plot as a high-resolution PNG file
plt.savefig("Observed_vs_Predicted_Test_Validation_Panel_fold2.png", format='png', dpi=300)

# Display the plot
plt.show()



# End of CV process

In [ ]:
import matplotlib.pyplot as plt

# Create the panel for both fold1 and fold2 in a single figure
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

# Plot for fold1
axes[0].plot(Tim.iloc[X_train_fold1.index], y_train_fold1, 'g', label='Training')
axes[0].plot(Tim.iloc[X_test_fold1.index], y_test_fold1, 'b', label='Testing')
axes[0].plot(Tim.iloc[X_val_fold1.index], y_val_fold1, 'black', label='Validation')
axes[0].set_ylabel('Water Level (WL_Mop)')
axes[0].legend(fontsize=8)
axes[0].text(0.99, 0.99, '(a)', transform=axes[0].transAxes, verticalalignment='top', horizontalalignment='right', fontsize=14)

# Plot for fold2
axes[1].plot(Tim.iloc[X_train_fold2.index], y_train_fold2, 'g', label='Training')
axes[1].plot(Tim.iloc[X_test_fold2.index], y_test_fold2, 'b', label='Testing')
axes[1].plot(Tim.iloc[X_val_fold2.index], y_val_fold2, 'black', label='Validation')
axes[1].set_ylabel('Water Level (WL_Mop)')
axes[1].legend(fontsize=8)
axes[1].text(0.99, 0.99, '(b)', transform=axes[1].transAxes, verticalalignment='top', horizontalalignment='right', fontsize=14)

# Adjust layout to avoid overlap
plt.tight_layout()

# Save the figure
plt.savefig("Training_Testing_Validation_Fold1_and_Fold2.png", format='png', dpi=300)

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import numpy as np
from matplotlib.ticker import FuncFormatter

# Set the font size for the whole plot
plt.rcParams.update({'font.size': 16})

# Create a figure with 3 rows and 4 columns: 1st row for test scatter plots, 2nd row for validation scatter plots, 3rd row for bar plots
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16, 12))

# Define the model names and their respective columns for scatter and bar plots
model_titles = ['RF', 'GB', 'SVM', 'XGB']
new_titles = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']
bar_titles = ['(i)', '(j)', '(k)', '(l)']

# Plot observed vs predicted values for test and validation sets (first 2 rows)
for i, (model_name, model) in enumerate(models_fold1.items()):
    # Test set plot (first row)
    ax_test = axes[0, i]
    test_predictions = model.predict(X_test_scaled_fold1)
    ax_test.scatter(y_test_fold1, test_predictions, alpha=0.4, color='blue', marker='o', s=10)
    ax_test.plot([y_test_fold1.min(), y_test_fold1.max()], [y_test_fold1.min(), y_test_fold1.max()], 'r--', lw=2)
    r2_test = r2_score(y_test_fold1, test_predictions)
    ax_test.text(0.95, 0.05, new_titles[i], horizontalalignment='right', verticalalignment='bottom', transform=ax_test.transAxes, fontsize=16, color='black')
    ax_test.text(0.05, 0.95, f'R²={r2_test:.2f}', transform=ax_test.transAxes, verticalalignment='top', color='blue', fontsize=16)
    ax_test.set_title(f'{model_titles[i]}', fontsize=16)
    if i == 0:
        ax_test.set_ylabel('Predicted water level (m)', fontsize=16)

    # Validation set plot (second row)
    ax_val = axes[1, i]
    val_predictions = model.predict(X_val_scaled_fold1)
    ax_val.scatter(y_val_fold1, val_predictions, alpha=0.4, color='black', marker='o', s=10)
    ax_val.plot([y_val_fold1.min(), y_val_fold1.max()], [y_val_fold1.min(), y_val_fold1.max()], 'r--', lw=2)
    r2_val = r2_score(y_val_fold1, val_predictions)
    ax_val.text(0.95, 0.05, new_titles[i + 4], horizontalalignment='right', verticalalignment='bottom', transform=ax_val.transAxes, fontsize=16, color='black')
    ax_val.text(0.05, 0.95, f'R²={r2_val:.2f}', transform=ax_val.transAxes, verticalalignment='top', color='black', fontsize=16)
    ax_val.set_xlabel('Observed water level (m)', fontsize=16)
    if i == 0:
        ax_val.set_ylabel('Predicted water level (m)', fontsize=16)

# Plot evaluation metrics (third row)
models = ['RF', 'GB', 'SVM', 'XGB']
bar_width = 0.35

# Values for bar plots (from your example)
rmse_test = combined_metrics_df_fold1['RMSE_Test']
rmse_val = combined_metrics_df_fold1['RMSE_Val']
mae_test = combined_metrics_df_fold1['MAE_Test']
mae_val = combined_metrics_df_fold1['MAE_Val']
mape_test = combined_metrics_df_fold1['MAPE_Test']
mape_val = combined_metrics_df_fold1['MAPE_Val']
nse_test = combined_metrics_df_fold1['NSE_Test']
nse_val = combined_metrics_df_fold1['NSE_Val']

# RMSE Bar Chart (first column)
axes[2, 0].bar(np.arange(len(models)), rmse_test, color='blue', width=bar_width, label='Test')
axes[2, 0].bar(np.arange(len(models)) + bar_width, rmse_val, color='black', width=bar_width, label='Val')
axes[2, 0].set_ylabel('RMSE (m)')
axes[2, 0].set_ylim(0.8, 1.8)
axes[2, 0].set_xticks(np.arange(len(models)) + bar_width / 2)
axes[2, 0].set_xticklabels(models)
axes[2, 0].text(0.95, 0.95, bar_titles[0], transform=axes[2, 0].transAxes, verticalalignment='top', horizontalalignment='right')
axes[2, 0].legend(loc='upper left', ncol=2, fontsize=10)

# MAE Bar Chart (second column)
axes[2, 1].bar(np.arange(len(models)), mae_test, color='blue', width=bar_width, label='Test')
axes[2, 1].bar(np.arange(len(models)) + bar_width, mae_val, color='black', width=bar_width, label='Val')
axes[2, 1].set_ylabel('MAE (m)')
axes[2, 1].set_ylim(0.6, 1.15)
axes[2, 1].set_xticks(np.arange(len(models)) + bar_width / 2)
axes[2, 1].set_xticklabels(models)
axes[2, 1].text(0.95, 0.95, bar_titles[1], transform=axes[2, 1].transAxes, verticalalignment='top', horizontalalignment='right')

# MAPE Bar Chart (third column)
axes[2, 2].bar(np.arange(len(models)), mape_test, color='blue', width=bar_width, label='Test')
axes[2, 2].bar(np.arange(len(models)) + bar_width, mape_val, color='black', width=bar_width, label='Val')
axes[2, 2].set_ylabel('MAPE')
axes[2, 2].set_ylim(0.4, 0.82)
axes[2, 2].yaxis.set_major_formatter(FuncFormatter(one_decimal))
axes[2, 2].set_xticks(np.arange(len(models)) + bar_width / 2)
axes[2, 2].set_xticklabels(models)
axes[2, 2].text(0.95, 0.95, bar_titles[2], transform=axes[2, 2].transAxes, verticalalignment='top', horizontalalignment='right')

# NSE Bar Chart (fourth column)
axes[2, 3].bar(np.arange(len(models)), nse_test, color='blue', width=bar_width, label='Test')
axes[2, 3].bar(np.arange(len(models)) + bar_width, nse_val, color='black', width=bar_width, label='Val')
axes[2, 3].set_ylabel('NSE')
axes[2, 3].set_ylim(0.3, 0.8)
axes[2, 3].yaxis.set_major_formatter(FuncFormatter(one_decimal))
axes[2, 3].set_xticks(np.arange(len(models)) + bar_width / 2)
axes[2, 3].set_xticklabels(models)
axes[2, 3].text(0.95, 0.95, bar_titles[3], transform=axes[2, 3].transAxes, verticalalignment='top', horizontalalignment='right')

# Adjust layout and show the final figure
plt.tight_layout()
plt.savefig("Combined_Scatter_Bar_Plots_Fold1.png", format='png', dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import numpy as np
from matplotlib.ticker import FuncFormatter

# Set the font size for the whole plot
plt.rcParams.update({'font.size': 16})

# Create a figure with 3 rows and 4 columns: 1st row for test scatter plots, 2nd row for validation scatter plots, 3rd row for bar plots
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16, 12))

# Define the model names and their respective columns for scatter and bar plots
model_titles = ['RF', 'GB', 'SVM', 'XGB']
new_titles = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)']
bar_titles = ['(i)', '(j)', '(k)', '(l)']

# Fit each model before making predictions
for model_name, model in models.items():
    model.fit(X_train_scaled_fold2, y_train_fold2)  # Train the model on training data

# Plot observed vs. predicted values for test and validation sets (first 2 rows)
for i, (model_name, model) in enumerate(models.items()):
    # Test set plot (first row)
    ax_test = axes[0, i]
    test_predictions = model.predict(X_test_scaled_fold2)
    ax_test.scatter(y_test_fold2, test_predictions, alpha=0.4, color='blue', marker='o', s=10)
    ax_test.plot([y_test_fold2.min(), y_test_fold2.max()], [y_test_fold2.min(), y_test_fold2.max()], 'r--', lw=2)
    r2_test = r2_score(y_test_fold2, test_predictions)
    ax_test.text(0.95, 0.05, new_titles[i], horizontalalignment='right', verticalalignment='bottom', transform=ax_test.transAxes, fontsize=16, color='black')
    ax_test.text(0.05, 0.95, f'R²={r2_test:.2f}', transform=ax_test.transAxes, verticalalignment='top', color='blue', fontsize=16)
    ax_test.set_title(f'{model_titles[i]}', fontsize=16)
    if i == 0:
        ax_test.set_ylabel('Predicted water level (m)', fontsize=16)

    # Validation set plot (second row)
    ax_val = axes[1, i]
    val_predictions = model.predict(X_val_scaled_fold2)
    ax_val.scatter(y_val_fold2, val_predictions, alpha=0.4, color='black', marker='o', s=10)
    ax_val.plot([y_val_fold2.min(), y_val_fold2.max()], [y_val_fold2.min(), y_val_fold2.max()], 'r--', lw=2)
    r2_val = r2_score(y_val_fold2, val_predictions)
    ax_val.text(0.95, 0.05, new_titles[i + 4], horizontalalignment='right', verticalalignment='bottom', transform=ax_val.transAxes, fontsize=16, color='black')
    ax_val.text(0.05, 0.95, f'R²={r2_val:.2f}', transform=ax_val.transAxes, verticalalignment='top', color='black', fontsize=16)
    ax_val.set_xlabel('Observed water level (m)', fontsize=16)
    if i == 0:
        ax_val.set_ylabel('Predicted water level (m)', fontsize=16)

# Define the metrics for the bar plots
models = ['RF', 'GB', 'SVM', 'XGB']
bar_width = 0.35

# Test Set results (from your example)
rmse_test = [0.39, 0.46, 0.37, 0.38]
mae_test = [0.31, 0.40, 0.28, 0.31]
mape_test = [0.24, 0.36, 0.20, 0.24]
nse_test = [0.93, 0.91, 0.94, 0.94]

# Validation Set results (from your example)
rmse_val = [0.41, 0.66, 0.52, 0.54]
mae_val = [0.31, 0.54, 0.41, 0.42]
mape_val = [0.20, 0.36, 0.29, 0.27]
nse_val = [0.96, 0.90, 0.94, 0.93]

# Define a function to format ticks to one decimal place
def one_decimal(x, pos):
    return f'{x:.1f}'

# RMSE Bar Chart (first column)
axes[2, 0].bar(np.arange(len(models)), rmse_test, color='blue', width=bar_width, label='Test')
axes[2, 0].bar(np.arange(len(models)) + bar_width, rmse_val, color='black', width=bar_width, label='Val')
axes[2, 0].set_ylabel('RMSE (m)')
axes[2, 0].set_ylim(0.3, 0.75)  # Adjust the y-axis range based on the RMSE values
axes[2, 0].set_xticks(np.arange(len(models)) + bar_width / 2)
axes[2, 0].set_xticklabels(models)
axes[2, 0].text(0.95, 0.95, bar_titles[0], transform=axes[2, 0].transAxes, verticalalignment='top', horizontalalignment='right')
axes[2, 0].legend(loc='upper left', ncol=2, fontsize=10)

# MAE Bar Chart (second column)
axes[2, 1].bar(np.arange(len(models)), mae_test, color='blue', width=bar_width, label='Test')
axes[2, 1].bar(np.arange(len(models)) + bar_width, mae_val, color='black', width=bar_width, label='Val')
axes[2, 1].set_ylabel('MAE (m)')
axes[2, 1].set_ylim(0.2, 0.6)  # Adjust the y-axis range based on the MAE values
axes[2, 1].set_xticks(np.arange(len(models)) + bar_width / 2)
axes[2, 1].set_xticklabels(models)
axes[2, 1].text(0.95, 0.95, bar_titles[1], transform=axes[2, 1].transAxes, verticalalignment='top', horizontalalignment='right')

# MAPE Bar Chart (third column)
axes[2, 2].bar(np.arange(len(models)), mape_test, color='blue', width=bar_width, label='Test')
axes[2, 2].bar(np.arange(len(models)) + bar_width, mape_val, color='black', width=bar_width, label='Val')
axes[2, 2].set_ylabel('MAPE')
axes[2, 2].set_ylim(0.1, 0.4)  # Adjust the y-axis range based on the MAPE values
axes[2, 2].yaxis.set_major_formatter(FuncFormatter(one_decimal))
axes[2, 2].set_xticks(np.arange(len(models)) + bar_width / 2)
axes[2, 2].set_xticklabels(models)
axes[2, 2].text(0.95, 0.95, bar_titles[2], transform=axes[2, 2].transAxes, verticalalignment='top', horizontalalignment='right')

# NSE Bar Chart (fourth column)
axes[2, 3].bar(np.arange(len(models)), nse_test, color='blue', width=bar_width, label='Test')
axes[2, 3].bar(np.arange(len(models)) + bar_width, nse_val, color='black', width=bar_width, label='Val')
axes[2, 3].set_ylabel('NSE')
axes[2, 3].set_ylim(0.7, 1)  # Adjust the y-axis range based on the NSE values
axes[2, 3].yaxis.set_major_formatter(FuncFormatter(one_decimal))
axes[2, 3].set_xticks(np.arange(len(models)) + bar_width / 2)
axes[2, 3].set_xticklabels(models)
axes[2, 3].text(0.95, 0.95, bar_titles[3], transform=axes[2, 3].transAxes, verticalalignment='top', horizontalalignment='right')

# Adjust layout and show the final figure
plt.tight_layout()
plt.savefig("Combined_Scatter_Bar_Plots_Fold2.png", format='png', dpi=300)
plt.show()
